# Q-learning algorithm

#### TO RUN ONCE (UTILS)

In [ ]:
# Load the classifiers
import joblib

file_path_vec = '/content/drive/MyDrive/tfidf_vectorizer_10000.joblib'
file_path_class = '/content/drive/MyDrive/random_forest_model_text.joblib'

vectorizer = joblib.load(file_path_vec)
classifier = joblib.load(file_path_class)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.1.3 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.1.3 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
# Load Sentiment Analysis RoBERTa model
# Run this only once in the entire notebook, if run twice it will provide an error. Interrup run time and rerun everything if needed

from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer_sentiment = AutoTokenizer.from_pretrained(MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
# Truncating excessive length of posts function
def truncate_doc(doc, max_len=200):
  tokens = doc.split()
  truncated_tokens = tokens[:min(max_len,len(tokens))]
  truncated_doc = " ".join(truncated_tokens)
  return truncated_doc


labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]
model1 = AutoModelForSequenceClassification.from_pretrained(MODEL)
model1.save_pretrained(MODEL)


# Defining the sentiment analysis function
def sentiment_analysis(text):
  res = []
  try:
    text = truncate_doc(text)
    encoded_input = tokenizer_sentiment(text, return_tensors='pt')
    output = model1(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    for i in range(scores.shape[0]):
      l = labels[ranking[i]]
      s = scores[ranking[i]]
      d = np.round(float(s),1)
      res.append(d)
  except:
    d = "Error length"
    res.append(d)

  return res

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
# Separate the lyrics into sentences
!pip install stanza
import stanza
stanza.download('en')
nlp = stanza.Pipeline('en')

def sentence_tokenizer(l):
  # Process the text
  doc = nlp(l)
  res = []
  for sentence in doc.sentences:
    res.append(' '.join([word.text for word in sentence.words]))
  print("done")
  return res

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 11.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Downloaded file to /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
# Create a Coherence score
import tensorflow_hub as hub
import tensorflow as tf
model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

def coherence_score(lyrics, new_lyrics):
  documents = [lyrics, new_lyrics]
  embeddings = model(documents)
  return np.inner(embeddings, embeddings)[0][1]


In [ ]:
# Load Gemma LLM
from tokenizers import Tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re

tokenizer2 = AutoTokenizer.from_pretrained("google/gemma-1.1-2b-it")
model2 = AutoModelForCausalLM.from_pretrained(
    "google/gemma-1.1-2b-it",
    torch_dtype=torch.bfloat16
  )

# Define the add function for Gemma, specifying the starting lyric for context and defining the input prompt
def to_add(lyric,index):
  start_index = index-2
  input_lyrics = lyric[start_index:index]
  rhyming_word = input_lyrics[-1].split()[-1]
  input_lyrics = " ".join(input_lyrics)
  input_length = len(input_lyrics.split())
  input_text = f'Write one line that continues this lyric "{input_lyrics}" and that gets closer to lyrics of top 100 billboard songs of the year 2000. The line must have lenght of {input_length//2} words and rhyme with the word {rhyming_word}'
  input_ids = tokenizer2(input_text, return_tensors="pt")
  encoded_output = model2.generate(**input_ids, max_new_tokens=30)
  output = tokenizer2.decode(encoded_output[0])
  pattern = r'\n\n\"(.*?)\.\"<eos>'
  m = re.search(pattern, output, re.DOTALL)
  lyrics = m.group(1).strip()
  return lyrics

# Define the modify function for Gemma, specifying the starting lyric for context and defining the input prompt
def to_modify(lyric,index):
  start_index = index-1
  end_index = index+1
  input_lyric = lyric[start_index:end_index]
  input_lyric[1] = f'[{input_lyric[1]}]'
  rhyming_word = input_lyric[0].split()[-1]
  input_length = len(input_lyric[1].split())
  input_lyric = " ".join(input_lyric)
  input_text = f'This is a lyric: "{input_lyric}".Write another lyric in place of the one in square brackets making it closer to lyrics of top 100 billboard songs. The line must have max lenght of {input_length} words and rhyme with the word {rhyming_word}.'
  input_ids = tokenizer2(input_text, return_tensors="pt")
  encoded_output = model2.generate(**input_ids, max_new_tokens=30)
  output = tokenizer2.decode(encoded_output[0])
  pattern = r'\n\n\"(.*?)\.\"<eos>'
  m = re.search(pattern, output, re.DOTALL)
  lyrics = m.group(1).strip()
  return lyrics

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

#### Data

In [ ]:
# Loading the data
import pandas as pd
data_all = pd.read_csv("/content/drive/MyDrive/Uni/total90_99.csv", on_bad_lines='skip')


In [ ]:
dataql = data_all[data_all.Label == 0].sample(n= 1500, random_state = 42)

In [ ]:
# Dividing the lyrics into lines
t  = [sentence_tokenizer(lyric) for lyric in dataql['lyrics'].tolist()]

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


In [ ]:
# Keep only if the lyric has a lenght greater than 6
lyrics_list = [i for i in t if len(i) > 6]

### Training

In [ ]:
# Defining the qlearning class:
import random
import itertools

class QLearning:
    def __init__(self, lyric, classifier, state_space, Q, num_actions=3):
        self.lyric_init = lyric
        self.lyric = self.lyric_init.copy()
        self.classifier = classifier
        self.state_space = state_space
        self.num_actions = num_actions
        # Initialize Q-table
        self.Q = Q
        self.sentiment, self.length = self.extract_features()
        self.state_index = self.get_state_index()
        self.current_score = self.get_reward()
        self.max_length = len(self.lyric)

    def extract_features(self):  # Extracting the features to determine the state
        lyrics = self.lyric
        sentiment = sentiment_analysis('\n'.join(lyrics))  # sentiment between 0 and 1 for neutral, negative and positive
        length = len(lyrics)  # Number of lines in the lyric
        return sentiment, length

    def get_state_index(self):  # Map features to a state index
        state = (self.sentiment, self.length)
        if state in self.state_space:
            return self.state_space.index(state)
        else:
            return None  # If the state is not found

    def get_reward(self):  # Define reward function
        lyric = self.lyric
        p = self.classify(lyric)
        p = np.mean(p[:, 1])
        old_lyric = ' '.join(self.lyric_init)
        new_lyric = ' '.join(lyric)
        c = coherence_score(old_lyric, new_lyric)
        return 0.9 * p + 0.1 * c

    def modify_line(self, lines, index): # Call to_modify Gemma
        modified_line = to_modify(lines,index)
        return lines[:index] + [modified_line] + lines[index+1:]

    def add_line(self, lines, index): # Call to_add Gemma
        new_line = to_add(lines,index)
        return lines[:index+1] + [new_line] + lines[index+1:]

    def remove_line(self, lines, index): # Remove a line
        return lines[:index] + lines[index+1:]

    def update_q(self, state_index, action_index, score_diff):  # Update Q-values with score difference
        if not isinstance(self.Q[state_index, action_index], list):
            self.Q[state_index, action_index] = []
        self.Q[state_index, action_index].append(score_diff)

    def run_episode(self):  # Run an episode of lyric modification
        try:
          action_index = random.randint(0, self.num_actions-1)
          line_index = random.randint(2, len(self.lyric)-2)
          if action_index == 1 and len(self.lyric) < self.max_length:  # Add
              self.lyric = self.add_line(self.lyric, line_index)
          elif action_index == 2 and len(self.lyric) > 4:  # Remove
              self.lyric = self.remove_line(self.lyric, line_index)
          else:  # Modify
              self.lyric = self.modify_line(self.lyric, line_index)
          new_score = self.get_reward()
          score_diff = new_score - self.current_score
          self.update_q(self.state_index, action_index, score_diff)
          self.sentiment, self.length = self.extract_features()
          self.state_index = self.get_state_index()
          self.current_score = new_score
        except Exception as e:
          print(e)

    def train_on_lyric(self, num_episodes=3):  # Function to train and modify the lyric
        final_lyrics = self.lyric[:]
        for _ in range(num_episodes):
            final_lyrics = self.run_episode()
        return self.Q

    def classify(self, lyrics): # Return the output of the classifier
        vectorized_lyrics = vectorizer.transform(lyrics)
        return classifier.predict_proba(vectorized_lyrics)

def q_train(lyrics_list, classifier, num_episodes=3): # Train function

    # Define state space
    num_actions = 3
    max_length = 1000
    sentiment_range = [round(x * 0.1, 1) for x in range(11)]
    state_combinations = [(list(fc), l) for fc in itertools.product(sentiment_range, repeat=3) for l in range(max_length + 1)]
    state_space = state_combinations
    Q = np.empty((len(state_space), num_actions), dtype=object)
    for i in range(len(state_space)):
        for j in range(num_actions):
          Q[i, j] = list()
    i = 0
    for lyric in lyrics_list:
        print(i)
        i += 1
        q_agent = QLearning(lyric, classifier, state_space, Q)
        Q = q_agent.train_on_lyric(num_episodes)
    return Q




# Run the training and get the average Q-table
Q = q_train(lyrics_list, classifier, num_episodes=3)

average_q_table = np.zeros(Q.shape)
for i in range(Q.shape[0]):
    for j in range(Q.shape[1]):
        if Q[i, j]:
            average_q_table[i, j] = np.mean(Q[i, j])
        else:
            average_q_table[i, j] = 0


print("Average Q-table:")
print(average_q_table)

In [ ]:
np.savetxt('qmatrix1.csv', average_q_table, delimiter=',', fmt='%d', header='Add,Modify,Remove', comments='')

### Deployment

In [ ]:
dataql.head()

,title,tag,artist,year,lyrics,Label
98863,Love in Vain,country,Ray Wylie Hubbard,1991,"Dreams die hard, the good die young Some are ...",0
124127,Bunkhouse Blues,country,Guy Clark,1999,At the Broken Heart Ranch You can always get w...,0
68202,Listen To The Waves,rock,Ayreon,1995,All the trees have fallen The soil is scorche...,0
171562,The Castle of Blackheim,rock,Diabolical Masquerade,1996,Somewhere beyond the frozen moors in the highe...,0
45225,New Miracle,pop,Bachelor Girl,1999,Where is this new miracle? Where is the christ...,0


In [ ]:
lyrics = dataql.lyrics.iloc[7] #input was reshuffled, these lyrics were not actually used in the training phase

In [ ]:
lyrics # Original lyrics

"Song, song for you  Oh, angel eyes are watching me tonight It makes me feel that it's\u2005all\u2005right I wanna be\u2005with you forever You're taking care for\u2005me every day No matter where I stay No matter where I go  You give me hope to carry on I know you are here tonight For you, my friend, I wrote this song I need you by my side  This is my song, this is my song for you I belong, yes, I belong to you This is my song, this is my song for you I belong, yes, I belong to you  Oh, angel eyes, oh, angel eyes You are the best friend in my life Once I will meet you in heaven I will sing this song And I will thank you for your love and your light You're always taking care of me No matter what I'm doin' You let your light shine down on me No matter where I go  This is my song, this is my song for you I belong (I belong) Yes, I belong to you This is my song (this is my song) This is my song for you I belong (I belong) Yes, I belong to you  You give me hope to carry on And I know you a

In [ ]:
Q = pd.read_csv("qmatrixnew.csv")

In [ ]:
states = Q['state'].tolist()
values = Q.drop('state', axis = 1)

In [ ]:
# Deployment Phase
import random

def modify_line(lines, index):
    modified_line = to_modify(lines,index)
    return lines[:index] + [modified_line] + lines[index+1:]

def add_line(lines, index):
    new_line = to_add(lines,index)
    return lines[:index+1] + [new_line] + lines[index+1:]

def remove_line(lines, index):
    return lines[:index] + lines[index+1:]

lyric_list = sentence_tokenizer(lyrics)
line_index = random.randint(2, len(lyric_list)-2)
for _ in range(3):
    sentiment = sentiment_analysis('\n'.join(lyric_list))
    length = len(lyric_list)
    state = (sentiment, length)
    print(f"({sentiment}, {length})")
    if state in states: #Choosing the action by max reward
        state_index = states.index(state)
        action = values.iloc[state_index].idxmax()
    else:
        action = random.choice([0, 1, 2]) #Choosing the action randomly, if the state is new
    if action == 0:
        lyric_list = add_line(lyric_list, line_index)
        print(f"Added a new line! (Sentiment: {sentiment}, Length: {length})")
    elif action == 1:
        lyric_list = modify_line(lyric_list, line_index)
        print(f"Modified a line! (Sentiment: {sentiment}, Length: {length})")
    elif action == 2:
        lyric_list = remove_line(lyric_list, line_index)
        print(f"Removed a line! (Sentiment: {sentiment}, Length: {length})")

done
([0.9, 0.1, 0.0], 28)
Removed a line! (Sentiment: [0.9, 0.1, 0.0], Length: 28)
([0.9, 0.1, 0.0], 27)
Removed a line! (Sentiment: [0.9, 0.1, 0.0], Length: 27)
([0.9, 0.1, 0.0], 26)
Modified a line! (Sentiment: [0.9, 0.1, 0.0], Length: 26)


In [ ]:
lyric_list #New modified lyrics

['Song , song for you Oh , angel eyes are watching me tonight',
 "It makes me feel that it 's all right I wan na be with you forever",
 "You 're taking care for me every day",
 'No matter where I stay',
 'No matter where I go You give me hope to carry on I know you are here tonight',
 'No matter where I go, You give me fire to ignite my soul',
 'You are the best friend in my life',
 'Once I will meet you in heaven I will sing this song',
 'And I will thank you for your love and your light',
 "You 're always taking care of me No matter what I 'm doin '",
 'You let your light shine down on me No matter where I go This is my song , this is my song for you I belong ( I belong )',
 'Yes , I belong to you',
 'This is my song ( this is my song )',
 'This is my song for you I belong ( I belong )',
 'Yes , I belong to you You give me hope to carry on And I know you are here tonight',
 'For you , my friend , I wrote this song I need you by my side',
 'This is my song , this is my song for you I 